## Machine Learning System for Equipment Failure Prediction

### A. System Overview

This machine learning system is designed to showcase an end-to-end implementation covering the entire lifecycle of a machine learning model. It integrates various MLOps tools and technologies discussed during the DASCI 270 sessions, that includes facilitating data ingestion, preprocessing, training, validation, deployment, and monitoring of the model. The system is also structured to handle drift detection to ensure the model remains effective and accurate over time. This document will guide you through interacting with the deployed **Equipment Failure Prediction** system, including making predictions, retrieving model information, and checking data drift, as well as provide detailed documentation of each component and their functionalities.

#### System Architecture

! insert diagram

#### Components

1. **Machine Learning Model (XGBoost for Classification)** - 

2. **Data Pipeline (Dagster)** - Orchestrates the workflow for data ingestion, preprocessing, and preparation. Dagster manages the sequence of these operations to ensure data flows correctly from one process to another, maintaining a clear and manageable execution order.

3. **Experiment Tracking (MLflow)** - Provides a framework to track experiments, including model training runs, parameters, metrics, and artifacts, enabling easier debugging and optimization. It stores models, performance metrics, and custom objects like drift reports, making them easy to access and compare across different runs.

4. **Model Serving (FastAPI)** - Deploys the trained model through a REST API using FastAPI, facilitating easy access to the model’s predictive capabilities. The API handles requests for predictions and provides model metadata, ensuring input validation and structured outputs.

5. **Containerization (Docker)** - Containerizes the services making up the system to ensure consistency and reproducibility across environments. Each isolated environment ("container") contains all necessary dependencies for each service, which can be easily deployed on any system supporting Docker.

6. **Drift Detection (Evidently AI)** - Integrates Evidently AI to monitor the model for any signs of data or concept drift. This component is crucial for maintaining the model's accuracy, providing insights into how the data characteristics and relationships are changing over time.

7. **Data Validation (Pydantic)** - Ensures that the data received by the API matches the expected format and type. This prevents errors during model prediction and ensures reliable model performance.

8. **Testing (Pytest and Github Actions)** - Uses Pytest to develop and run comprehensive tests that validate the correctness of the data processing and feature engineering components. GitHub Actions automates these tests, ensuring that all code integrations meet quality standards and function as expected.

#### Dataset

This project utilized the AI4I 2020 Predictive Maintenance dataset, a synthetic dataset sourced from UCI Machine Learning Repository: https://archive.ics.uci.edu/dataset/601/ai4i+2020+predictive+maintenance+dataset. It contains 10,000 data points with the following columns/variables:

| Variables          | Type        | Description                                                                                         |
|--------------------|-------------|-----------------------------------------------------------------------------------------------------|
| UID                | Integer     | Unique identifier ranging from 1 to 10000                                                           |
| Product ID         | Categorical | Consists of a letter (L, M, H) for low, medium, and high product quality variants with a serial number |
| Type               | Categorical | Not specified                                                                                       |
| Air temperature    | Continuous  | Generated using a random walk process, normalized around 300 K with a standard deviation of 2 K      |
| Process temperature| Continuous  | Generated using a random walk process, normalized to a standard deviation of 1 K, plus air temperature + 10 K |
| Rotational speed   | Integer     | Calculated based on a power of 2860 W with normally distributed noise                               |
| Torque             | Continuous  | Normally distributed around 40 Nm with a standard deviation of 10 Nm, no negative values             |
| Tool wear          | Integer     | Tool wear added by quality variants H/M/L are 5/3/2 minutes respectively                             |
| Machine failure    | Integer     | Indicates if the machine has failed (1) or not (0) in this particular datapoint                      |
| TWF                | Integer     | Target feature indicating if a specific type of failure occurred or not (1/0)                        |


#### Data Preprocessing



### B. Setup Instructions

This section provides detailed instructions on how to set up and run the system using Docker Compose. The instructions assume you have Docker and Docker Compose installed on your machine. If not, please install them from the [Docker official website](https://www.docker.com/) before proceeding.

**Step 1: Clone the Repository** <br>
First, clone the project repository from GitHub to get the necessary code and configuration files. Use the following command:

```python
git clone https://github.com/raymundojavajr/ml-fp.git
cd ml-fp
```

This command clones the repository into a directory named ml-fp on your local machine and changes into that directory.

**Step 2: Configure Environment Variables** <br>
Create a .venv file in the root directory of the project to store environment variables. This can be conveniently done using uv:

```python
uv sync
```

It can also be done using pip:

```python
python -m .venv venv
pip install -r requirements.txt
```

**List of Libraries used in the pipeline:**



**Step 3: Build the Docker Containers**
Use Docker Compose to build the services defined in the docker-compose.yml file. This includes your Dagster orchestration, MLflow tracking server, and FastAPI application. Run the following command:

```python
docker-compose build
```

This command builds the Docker images for each service according to the specifications in the Dockerfile and docker-compose.yml files. It installs all necessary dependencies in these images, ensuring each service has what it needs to run.

**Step 4: Run the Services**
Once the images are built, you can start the services with the following command:

```python
docker-compose up
```

This command starts all services defined in docker-compose.yml. It creates and starts Docker containers for each service. The services will be networked together as defined, allowing them to communicate with each other.

**Step 5: Access the Services**
* FastAPI Application: Access the FastAPI application at http://localhost:8000. You will find the automatically generated API documentation there, which allows you to interact with the API directly through your web browser.
* MLflow Tracking Server: Access the MLflow tracking server at http://localhost:5000. Here you can view and compare different model runs and their metrics.
* Dagster Dagit UI: Access the Dagster orchestration UI at http://localhost:3000. This interface allows you to monitor and control the data pipelines.

**Step 6: Shut Down the Services**
To stop and remove all running containers, use the following command:

```python
docker-compose down
```
This command stops all containers and removes the containers, networks, and volumes created by docker-compose up. It cleans up everything except the built images, allowing for a quick restart of the services if needed.



### C. Happy Path

#### Prediction Request

In this section, we demonstrate how to use the FastAPI model server for predictions. We'll send a set of equipment parameters and status data to receive predictions on potential machine failures.

- **Input Data:** This includes a series of variables such as equipment type, air and process temperatures, rotational speed, torque, tool wear, and previous instances of machine failure.
- **API Endpoint:** This is the URL where the FastAPI server accepts prediction requests.
- **Response Handling:** Upon receiving the prediction request, the server provides a forecast indicating whether a machine is likely to fail. This prediction is displayed in the output.

Execute the following cell to initiate a prediction request.

In [ ]:
import requests

# FastAPI server URL
FASTAPI_URL = "http://localhost:8000/predict"

# Define the input data
# In order of the features: UDI, Air temperature (K), Process temperature (K), Rotational speed (rpm), Torque (Nm), Tool wear (min), Type encoded, Product ID encoded, Failure Type encoded
# Ensure to adjust the data based on actual possible inputs and features expected by the model
payload = {
    "features": [
        2,          # UDI
        310.5,      # Air temperature in Kelvin
        320.1,      # Process temperature in Kelvin
        2000,       # Rotational speed in rpm
        50.5,       # Torque in Nm
        10,         # Tool wear in minutes
        1,          # Type encoded (categorical, numeric encoding)
        8005,       # Product ID encoded (categorical, numeric encoding)
        0           # Failure Type encoded (target variable, if used for retraining or similar scenarios)
    ]
}

# Send request to FastAPI
response = requests.post(FASTAPI_URL, json=payload)

# Handle response
if response.status_code == 200:
    print("Prediction:", response.json())  # Print the predicted output, likely a machine failure prediction or similar
else:
    print("Error:", response.status_code, response.text)  # Handle errors like 404 or 500 from server

#### Model Information Retrieval



In [ ]:
# import requests

# # FastAPI server URL for the /model endpoint
# FASTAPI_URL = "http://localhost:8000/model"

# # Send request to FastAPI
# response = requests.get(FASTAPI_URL)

# # Handle response
# if response.status_code == 200:
#     model_info = response.json()
#     print("Model Information:")
#     print("Hyperparameters:", model_info["hyperparameters"])
#     print("Important Features:", model_info["important_features"])
#     print("Input Schema:", model_info["input_schema"])
# else:
#     print("Error:", response.status_code, response.text)

### D. Drift Detection Demonstration

#### Evidently AI Access from MLflow

Insert instructions and demonstration on how to access the Evidently AI drift reports from MLflow

Show Python code to access MLflow and retrieve the reports

Show screenshots or directly embed the HTML reports

#### Drift Result Analysis

### E. Reproducability

### F. References